In [1]:
import pandas as pd

negative_set = pd.read_csv('negative_set.bed', delimiter='\t', header = None)
gm = pd.read_csv('GM12878.bed', delimiter='\t', header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'negative_set.bed'

In [7]:
negative_set.head()

,0,1,2
0,chr10,32251,36771
1,chr10,39431,39891
2,chr10,72312,74222
3,chr10,84717,85177
4,chr10,90499,91949


In [8]:
gm.head()

,0,1,2,3
0,chr1,773300,774100,7.866088
1,chr1,778980,779450,6.472419
2,chr1,800100,802000,11.010675
3,chr1,825670,826410,6.114487
4,chr1,839470,842590,8.848865


In [18]:
GRCh37 = pd.read_csv('/Users/zuzannaglinka/Documents/Studia/WB/GRCh37.primary_assembly.genome.fa', sep='\t')

In [19]:
GRCh37

,>chr1 1
0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
2,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
3,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
4,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
...,...
51696864,GTTGTAGTGCTGGCTGTTGTTGTGGTTGTGGGTTCAGTAGAGGTGG...
51696865,GTGGTTGTGGTCTCAGTAGCAGTGGCTGCAGTTGTGGTGGCAGACT...
51696866,CTTGCTGTGGTTGTCATTGTGGGTTCAGCAGAAGTGGCAGTGGTGG...
51696867,GTCTCAGTGGAAGTGCCTGCAGTTGTGGTGGCAGACTGTTCAGTTG...
